In [1]:
pip install tensorflow scikit-learn numpy


In [2]:
import zipfile
import os

def unzip_file(zip_file, extract_dir):
    # Check if the target directory exists, if not create it
    if not os.path.exists(extract_dir):
        os.makedirs(extract_dir)

    # Open the zip file
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        # Extract all the contents into the target directory
        zip_ref.extractall(extract_dir)

# Example usage
zip_file = 'kidney_ct_scan_data.zip'
extract_dir = 'kidney_ct_scan_data'

unzip_file(zip_file, extract_dir)


In [3]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define data directories
train_dir = '/content/kidney_ct_scan_data/kidney_ct_scan_data/train'
test_dir = '/content/kidney_ct_scan_data/kidney_ct_scan_data/test'

# Image dimensions
img_width, img_height = 224, 224

# Rescale images and create data generators
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary')


Found 341 images belonging to 2 classes.
Found 124 images belonging to 2 classes.


In [4]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model

# Load the VGG16 model without the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze the pre-trained weights
for layer in base_model.layers:
    layer.trainable = False

# Add custom top layers
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


58889256/58889256 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

In [6]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size)


Epoch 1/100
10/10 [==============================] - 6s 683ms/step - loss: 0.2664 - accuracy: 0.9417 - val_loss: 4.5615 - val_accuracy: 0.7396
Epoch 2/100
10/10 [==============================] - 2s 214ms/step - loss: 1.4394e-04 - accuracy: 1.0000 - val_loss: 5.2149 - val_accuracy: 0.6875
Epoch 3/100
10/10 [==============================] - 2s 210ms/step - loss: 0.0026 - accuracy: 0.9968 - val_loss: 5.7376 - val_accuracy: 0.7083
Epoch 4/100
10/10 [==============================] - 3s 312ms/step - loss: 2.2730e-04 - accuracy: 1.0000 - val_loss: 5.2714 - val_accuracy: 0.7396
Epoch 5/100
10/10 [==============================] - 2s 191ms/step - loss: 2.0540e-04 - accuracy: 1.0000 - val_loss: 4.3235 - val_accuracy: 0.7917
Epoch 6/100
10/10 [==============================] - 2s 199ms/step - loss: 5.7735e-04 - accuracy: 1.0000 - val_loss: 6.8046 - val_accuracy: 0.6667
Epoch 7/100
10/10 [==============================] - 2s 211ms/step - loss: 2.2271e-05 - accuracy: 1.0000 - val_loss: 6.0610 - 

In [7]:
score = model.evaluate(validation_generator, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Test loss: 5.140483856201172
Test accuracy: 0.7177419066429138


In [8]:
model.save('model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [12]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

In [16]:
import cv2

def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))  # Resize the image to match the input shape of the model
    img = np.expand_dims(img, axis=0)
    return img

# Path to the input image you want to predict
input_image_path = '/content/kidney_ct_scan_data/kidney_ct_scan_data/test/Normal/Normal- (965).jpg'

# Load your trained model
model = load_model('model.h5')

# Preprocess the input image
input_image = preprocess_image(input_image_path)

# Make prediction
prediction = model.predict(input_image)

# Interpret prediction
if prediction < 0.5:
    print("Prediction: Normal")
else:
    print("Prediction: Tumor")


1/1 [==============================] - 0s 131ms/step
Prediction: Normal
